# Day04

今回は`pandas`を使用した、データの前処理をおこなう。  
pandasではデータフレームと呼ばれるオブジェクトをもとに様々なデータの処理を行うことができる。  

## 欠損値の処理
まず、欠損値を含んだ適当なデータフレームを作成する。

In [1]:
import pandas as pd
from io import StringIO

# 適当なデータフレームを作成

csv_data = \
'''
A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,
'''

# StringIO(csv_data)はcsv_dataが書かれたcsvファイルとして扱われる。
df = pd.read_csv(StringIO(csv_data))

df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


### 欠損値の確認
`df.isnull()`で欠損値が`True`そうでない箇所が`False`になったデータフレームが得られる。  
それを`sum()`メソッドで論理和を取って、表示するイメージ。

In [2]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### 欠損値のある行/列を削除
欠損値のある行または列を削除するには`dropna`メソッドを使用する。  
引数に`axis=0`を指定した場合は行方向、`axis=1`を指定した場合は列方向に処理される。

In [3]:
# 行方向
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [4]:
# 列方向
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [5]:
# 特定のカラムの欠損値のみを見て、行の削除も可能
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 欠損値の保管
平均値や最頻値から欠損値を埋めることができる。  
しかし、この操作はデータの改変になるためkaggleなどでは推奨しない。

In [6]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


## カテゴリデータの整形

ここでも、適当なデータフレームを作成し、それをもとに解説を行う。

In [7]:
import pandas as pd

# カテゴリデータを含むデータフレームを作成
df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### 順序特徴量
XL, L, M などのカテゴリデータは大小関係が決まっているので、数値に変換することが可能である。  
ここでは、変換表を作成し、それをもとに変換する方法を紹介する。

In [8]:
# 変換するための辞書
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


### 名義特徴量
`color`などのカテゴリデータはデータ感に順序が存在しない。  
なので、データ間の距離が常に一定のone-hot表現に変換する必要がある。  
pandasのget_dummiesメソッドを利用すると簡単に名義特徴量をone-hot表現に変換できる。

In [9]:
# one-hot 表現に変換
pd.get_dummies(df[['price', 'color', 'size']], dtype=int)

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


また、one-hot表現はカラムを一つ削除しても値が削除した値が逆算できるので次元を削減することができる。  
その際には引数に`drop_first=True`を追加する。

In [10]:
pd.get_dummies(df[['price', 'color']], dtype=int, drop_first=True)

,price,color_green,color_red
0,10.1,1,0
1,13.5,0,1
2,15.3,0,0


データフレームを変換するには得られたone-hot表現を結合しもとのカラムを削除すれば良い

In [11]:
# one-hot表現を結合
onehot = pd.get_dummies(df[['color']], dtype=int)
df = pd.concat([df, onehot], axis=1)

df = df.drop('color', axis=1)

df

,size,price,classlabel,color_blue,color_green,color_red
0,1,10.1,class2,0,1,0
1,2,13.5,class1,0,0,1
2,3,15.3,class2,1,0,0


## 正則化

### wine dataset
これまでは Iris Dataset を例に説明してきたが、モデルを複雑にするため、 wine dataset を利用する。  
wine dataset はイタリアの同じ地域で栽培されたワインを化学的に分析した特徴量と、どのワインかのラベルで構成される特徴量である。  
特徴量は以下の通り。
- alcohol： アルコール度数
- malic_acid： リンゴ酸
- ash： 灰分（かいぶん）
- alcalinity_of_ash： 灰分のアルカリ度
- magnesium： マグネシウム
- total_phenols： 全フェノール含量
- flavanoids： フラボノイド
- nonflavanoid_phenols： 非フラボノイドフェノール
- proanthocyanins： プロアントシアニン
- color_intensity： 色の濃さ
- hue： 色相
- od280/od315_of_diluted_wines： 希釈ワイン溶液の吸光度の比
- proline： プロリン

In [12]:
import numpy as np
import pandas as pd

# インターネット上のcsvからデータフレームを作成
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

# カラム名を設定
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

df_wine.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


現在、wine dataset はデータフレームになっているのでそれを計算するため、numpyのndarrayとして取り出す。  
データフレームからデータを取り出すには `iloc` メソッドを使用しする。  
ここで、`iloc[:, 1:]`はすべての行を取り出し、1列目移行(クラスラベル以外)の列を取り出すという意味。  
それによって、取り出された特徴量は`pandas.Siries`なのでそれを`ndarray`として取り出すには、`value`アトリビュートにアクセスする。  
取り出しが完了したら、データを分割して、標準化しておく。

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.3, 
                     random_state=0, 
                     stratify=y)

stdsc = StandardScaler()
X_train = stdsc.fit_transform(X_train)
X_test = stdsc.transform(X_test)

### L1正則化

ロジスティック回帰モデルにL1正則化を実施してみる。
L1の正則化を施すにはを引数、`penalty='l1'`を指定する。ここで、`C=1.0`は正則化パラメータの強さを意味する。

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l1', C=1.0, solver='liblinear', multi_class='ovr')

lr.fit(X_train, y_train)
print('Training accuracy:', lr.score(X_train, y_train))
print('Test accuracy:', lr.score(X_test, y_test))

Training accuracy: 1.0
Test accuracy: 1.0


ロジスティック回帰の重みの値を表示してみると多くの重みを0にして、モデルの複雑化を防いでいることがわかる。

In [15]:
# L1 正則化後のロジスティック回帰モデルの重み
lr.coef_

array([[ 1.2461223 ,  0.18076088,  0.74337296, -1.16118892,  0.        ,
         0.        ,  1.17070512,  0.        ,  0.        ,  0.        ,
         0.        ,  0.54642758,  2.51030577],
       [-1.53726799, -0.38699877, -0.99526852,  0.36484939, -0.05959667,
         0.        ,  0.66808871,  0.        ,  0.        , -1.93396495,
         1.23382235,  0.        , -2.23190508],
       [ 0.13547348,  0.1686736 ,  0.35718618,  0.        ,  0.        ,
         0.        , -2.43761566,  0.        ,  0.        ,  1.56378882,
        -0.81910926, -0.49265753,  0.        ]])

### L2正則化

ロジスティック回帰モデルにL2正則化を実施してみる。
L1の正則化を施すにはを引数、`penalty='l2'`を指定する。ここで、`C=1.0`は正則化パラメータの強さを意味する。

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', multi_class='ovr')

lr.fit(X_train, y_train)
print('Training accuracy:', lr.score(X_train, y_train))
print('Test accuracy:', lr.score(X_test, y_test))

Training accuracy: 0.9919354838709677
Test accuracy: 1.0


正則化を行っていないモデルと比べて重みの大きさを小さくし、モデルの複雑化を防いでいることがわかる。

In [17]:
# L1 正則化後のロジスティック回帰モデルの重み
lr.coef_

array([[ 1.27713853,  0.38210274,  0.8015599 , -1.30842842,  0.22782837,
         0.23101419,  0.90234371, -0.08423823,  0.01462196, -0.0312838 ,
         0.02796323,  0.71703048,  1.79262118],
       [-1.45395571, -0.620303  , -1.05445248,  0.67148394, -0.29048951,
         0.18277571,  0.51163918,  0.10789643,  0.08199321, -1.61228834,
         0.88800662,  0.1659356 , -1.73246957],
       [ 0.38965148,  0.4083047 ,  0.40211468,  0.26242969,  0.15288658,
        -0.20064653, -1.38792256, -0.06305419, -0.28440345,  1.2553389 ,
        -0.93849662, -0.83821807,  0.13754706]])

In [18]:
# 正則化をしていないロジスティック回帰モデルの重み

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty=None, multi_class='ovr')

lr.fit(X_train, y_train)

lr.coef_

array([[  9.71129297,   3.88265195,   7.62566851, -11.03851514,
          2.75480157,  -0.87679649,   6.31253312,  -1.05319829,
         -0.1581623 ,  -2.30097528,  -1.21296895,   6.88935381,
         10.24814278],
       [-14.30899893,  -5.3394532 , -16.4083766 ,   8.98999651,
          2.31860883,  -1.55878013,  11.24981592,   6.76786745,
         -2.26504953, -26.25428738,  13.30439872,   0.60758132,
        -26.50433674],
       [  6.49764496,   0.80783328,   4.95554199,   3.06499224,
         -2.52859193,   2.74619654, -15.93347064,  -4.94930726,
         -3.39979845,  10.78476565,  -9.15049332,  -6.76212839,
          4.36759536]])